# Skin Cancer MNIST or Dermatology MNIST
## Loading and Processing

In [1]:
#Import the necessary modules
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import glob,os
from sklearn.model_selection import train_test_split

Note: The dataset can be obatained from kaggle datatsets by searching for Skin Cancer MNIST:HAM10000. Link : www.kaggle.com/kmader/skin-cancer-mnist-ham10000 .

In [2]:
#The image directory where images are stored. The zips in dataset were extracted to the specified folder.
data_dir = 'data/SkinMNIST/Images/'

In [3]:
#There are 7 types of classes in the dataset for lesions as specified:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [4]:
#Reading the data from the csv file and saving it as a dataframe.
df_original = pd.read_csv('data/SkinMNIST/HAM10000_metadata.csv')


#Add columns to the original DataFrame, cell_type (the whole name),cell_type_idx (the corresponding index of cell type, as the image label.
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes #Also Convert each lesion to  a numerical code.
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign keratosis-like lesions,2


Now we are required to have a training dataset and a testing dataset. But we need to check for duplicates in our dataset as those are not reuired in our validation set.

In [5]:
#Creating a new dataframe df_undup that contains only the non-duplicate elements.
df_undup = df_original.groupby('lesion_id').count()
#Filters out so that we have only one image associated with each lesion_id
df_undup = df_undup[df_undup['image_id'] == 1]
df_undup.reset_index(inplace=True)
df_undup.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,cell_type_idx
0,HAM_0000001,1,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1,1


Here we create a new column that specifies whether a lesion_id is duplicated or not.

In [6]:
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'
df_original['duplicates'] = df_original['lesion_id']
df_original['duplicates'] = df_original['duplicates'].apply(get_duplicates)
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,cell_type,cell_type_idx,duplicates
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign keratosis-like lesions,2,duplicated
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign keratosis-like lesions,2,duplicated


In [7]:
#Checking the number of duplicates.
df_original['duplicates'].value_counts()

unduplicated    5514
duplicated      4501
Name: duplicates, dtype: int64

In [8]:
#creating the dataframe with only non-duplicate elements.
df_undup = df_original[df_original['duplicates'] == 'unduplicated']
df_undup.shape

(5514, 10)

In [9]:
#Creation of a validation set by srandomly choosing 20% rows from non-duplicate dataset.
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

(1103, 10)

Now we are required to create our training dataset which can contain the duplicate elements but non the validation ones.

In [10]:
def get_val_rows(x): #To check whether a row is present in validation or not.
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# create a new colum that is a copy of the image_id column
df_original['train_or_val'] = df_original['image_id']
# apply the function to this new column
df_original['train_or_val'] = df_original['train_or_val'].apply(get_val_rows)
# filter out train rows
df_train = df_original[df_original['train_or_val'] == 'train']
print(len(df_train))

8912


Now we have got our training and validation dataset.

Lets view the distribution in train dataset.

In [11]:
df_train['cell_type_idx'].value_counts()

4    5822
6    1067
2    1011
1     479
0     297
5     129
3     107
Name: cell_type_idx, dtype: int64

As train dataset is very skewed  and unbalanced, so we augment it to even it out and have a uniform distribution.

In [12]:
data_aug_rate = [15,10,5,50,0,40,5] #Setting augmentation rates according to ratio of examples in dataset.
for i in range(7):
    if data_aug_rate[i]:
        df_train=df_train.append([df_train.loc[df_train['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
df_train['cell_type_idx'].value_counts()

4    5822
3    5350
6    5335
5    5160
2    5055
1    4790
0    4455
Name: cell_type_idx, dtype: int64

So now we have got our training dataset with evenly distributed data.

In [13]:
df_train = df_train.reset_index()
df_val = df_val.reset_index()

In [14]:
#The mean and standard values of images for normalisation
#These are to be chosen according to the model we are going to use.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

### Creating train and testing Dataloaders.

In [15]:
#Here we define the transforms that are to be applied to images to provide better generalization.
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(30),
                                       transforms.ToTensor(),
                                       transforms.Normalize(norm_mean, norm_std)])

In [16]:
val_transforms = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(norm_mean, norm_std)])

In [17]:
#The class is created so that image and its associated label can be combined in a single dataset.
class createDataset (Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        a=self.df['image_id'][index]
        images = Image.open(data_dir +a+'.jpg')
        labels= torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            images = self.transform(images)

        return images, labels

In [18]:
#Creating the training dataset.
train_dataset = createDataset(df_train, transform=train_transforms)

In [19]:
train_dataset.__getitem__(0)

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [20]:
#Creating the generator / DataLoader for training dataset.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [21]:
#Creating the validation dataset.
val_dataset = createDataset(df_val , transform = val_transforms)

In [22]:
#Creating the generator/DataLoader for the validation dataset.
val_loader = DataLoader(val_dataset,batch_size=32, shuffle=False)

In [23]:
val_dataset.__getitem__(0)

(tensor([[[ 1.5639,  1.5982,  1.6324,  ...,  1.4612,  1.2557,  0.8789],
          [ 1.5982,  1.6324,  1.6495,  ...,  1.4954,  1.3242,  0.8961],
          [ 1.5982,  1.6153,  1.6324,  ...,  1.4783,  1.3755,  1.0159],
          ...,
          [ 1.3242,  1.3242,  1.3242,  ...,  1.4440,  1.4440,  1.2728],
          [ 1.3242,  1.3242,  1.2899,  ...,  1.4269,  1.4269,  1.3070],
          [ 1.3070,  1.3242,  1.3070,  ...,  1.3584,  1.3927,  1.3242]],
 
         [[ 0.4153,  0.4503,  0.4503,  ...,  0.3452,  0.1702, -0.1800],
          [ 0.3978,  0.4153,  0.4503,  ...,  0.3452,  0.2402, -0.1450],
          [ 0.3803,  0.3978,  0.4153,  ...,  0.3452,  0.3102, -0.0574],
          ...,
          [ 0.3102,  0.3277,  0.2752,  ...,  0.6779,  0.6429,  0.4328],
          [ 0.3277,  0.3102,  0.2577,  ...,  0.6779,  0.6604,  0.4853],
          [ 0.3627,  0.3277,  0.2752,  ...,  0.6078,  0.5903,  0.5028]],
 
         [[ 0.7402,  0.7751,  0.8099,  ...,  0.5485,  0.2696, -0.1661],
          [ 0.7228,  0.7751,

## Model building, training and evaluation.

As we are using transfer learning, here we use the Dense121 model taht is already provided by PyTorch. The model contains 121 interconnected layers. We can download the pretrained model directly for use with the help of torchvision.models. For more details to densenet, refer: https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py 

In [24]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

So after looking at the model it has 2 parts, features and classifier . The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. But the classifier takes 1024 inputs and outputs 1000 features. As this is different to what we are required we need to redefine the classifier.

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Check if the machine supports cuda.


for param in model.parameters(): #Here we freeze the parameters. i.e they wont be updated.
    param.requires_grad = False  #They can be unfrozen later to fine tune them too.
    

#Now we define our own classifier with 1024 inputs and 7 outputs.
#Here the ReLU and Log_softmax activation functions are used.
#To avoid overfitting, dropout is set to 0.2.
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 7),
                                 nn.LogSoftmax(dim=1))

#Here we define our loss function, which in this case is negative log-likelihood loss.
criterion = nn.NLLLoss()

#We define our optimizer.Here the ADAM optimzer is used with learning rate= 0.001. SGD can also be used in its place.
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

#Move the model to default device.
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

Now as we have defined our model we can training it and evaluating its performance.

In [ ]:
epochs = 10 #Define the epochs for which the model is to be run.
steps = 0
running_loss = 0
print_every = 10 #VAriable set to define to print loss and accuracy after every 'print_every' no of steps/batches.
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval() 
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(val_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(val_loader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/10.. Train loss: 1.890.. Test loss: 1.742.. Test accuracy: 0.129
Epoch 1/10.. Train loss: 1.722.. Test loss: 1.286.. Test accuracy: 0.737
Epoch 1/10.. Train loss: 1.567.. Test loss: 1.586.. Test accuracy: 0.444
Epoch 1/10.. Train loss: 1.456.. Test loss: 1.193.. Test accuracy: 0.621
Epoch 1/10.. Train loss: 1.324.. Test loss: 1.143.. Test accuracy: 0.657
Epoch 1/10.. Train loss: 1.397.. Test loss: 1.073.. Test accuracy: 0.708
Epoch 1/10.. Train loss: 1.294.. Test loss: 0.760.. Test accuracy: 0.792
Epoch 1/10.. Train loss: 1.292.. Test loss: 1.148.. Test accuracy: 0.628
Epoch 1/10.. Train loss: 1.319.. Test loss: 0.911.. Test accuracy: 0.722
Epoch 1/10.. Train loss: 1.190.. Test loss: 0.993.. Test accuracy: 0.696
Epoch 1/10.. Train loss: 1.218.. Test loss: 0.850.. Test accuracy: 0.727
Epoch 1/10.. Train loss: 1.122.. Test loss: 0.736.. Test accuracy: 0.777
Epoch 1/10.. Train loss: 1.277.. Test loss: 0.811.. Test accuracy: 0.726
Epoch 1/10.. Train loss: 1.203.. Test loss: 0.709..

Epoch 2/10.. Train loss: 0.710.. Test loss: 0.556.. Test accuracy: 0.792
Epoch 2/10.. Train loss: 0.782.. Test loss: 0.472.. Test accuracy: 0.830
Epoch 2/10.. Train loss: 0.797.. Test loss: 0.730.. Test accuracy: 0.720
Epoch 2/10.. Train loss: 0.874.. Test loss: 0.432.. Test accuracy: 0.851
Epoch 2/10.. Train loss: 0.709.. Test loss: 0.514.. Test accuracy: 0.809
Epoch 2/10.. Train loss: 0.844.. Test loss: 0.512.. Test accuracy: 0.808
Epoch 2/10.. Train loss: 0.820.. Test loss: 0.506.. Test accuracy: 0.816
Epoch 2/10.. Train loss: 0.860.. Test loss: 0.444.. Test accuracy: 0.849
Epoch 2/10.. Train loss: 0.801.. Test loss: 0.484.. Test accuracy: 0.825
Epoch 2/10.. Train loss: 0.816.. Test loss: 0.456.. Test accuracy: 0.836
Epoch 2/10.. Train loss: 0.815.. Test loss: 0.553.. Test accuracy: 0.797
Epoch 2/10.. Train loss: 0.843.. Test loss: 0.514.. Test accuracy: 0.813
Epoch 2/10.. Train loss: 0.858.. Test loss: 0.495.. Test accuracy: 0.824
Epoch 2/10.. Train loss: 0.788.. Test loss: 0.471..

Note: Here the training was not completed due to limitations of the local machine. But as it can be seen even after one epoch the model was able to achieve the validation accuracy of 80.7%. When trained on a different machine the model was able to achieve an validation accuracy of 90% in 10-15 epochs.

### Final Words

In this case the model was tested with parameters of densenet model in the frozen state. So weights of layers of the densenet model were not being updated. The layers of the densenet can be unfrozen and the training can be re-run.
The model is able to achieve validation accuracy of equals or more than 90% after 10-15 epochs.